<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-Projects/blob/main/Food-Receipe-Text-Generation(Recurrent%20Networks)/food_receipe_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [6]:
!pip install -q datasets==3.6.0

In [7]:
import tensorflow as tf
import numpy as np
import os
import re

import datasets

# Get Dataset

In [8]:
# Configuration
MAX_TOKENS = 10_000       # Vocabulary size (ingredients are diverse)
SEQ_LENGTH = 60           # Context window (recipes are long)
EMBEDDING_DIM = 256
RNN_UNITS = 512
BATCH_SIZE = 64
EPOCHS = 50

# Training Pipeline

In [9]:
def get_dataset():
    return datasets.load_dataset("m3hrdadfi/recipe_nlg_lite",
                                 trust_remote_code=True)

def format_recipe(input):
    name = input['name'] if input['name'] else 'unkown'
    ingredients = input['ingredients'] if input['ingredients'] else ''
    steps = input['steps'] if input['steps'] else ""

    text = f"<START> title: {name} <ING> {ingredients} <DIR> {steps} <END>"
    return text

@tf.keras.utils.register_keras_serializable()
def receipe_standardization(input):
    text = tf.strings.lower(input)
    text = tf.strings.regex_replace(text, f"(<start>|<ing>|<dir>|<end>)", r" \1 ")
    text = tf.strings.regex_replace(text, r"([.,!?()])", r" \1 ")
    text = tf.strings.regex_replace(text, r"[^a-zA-Z0-9.,!?()<>: ]", "")
    text = tf.strings.regex_replace(text, r"\s{2,}", " ")
    return text

def create_dataset_pipeline(text_corpus):
    vectorizer = tf.keras.layers.TextVectorization(
        standardize=receipe_standardization,
        max_tokens=MAX_TOKENS,
        output_mode="int",
        output_sequence_length=None
    )
    vectorizer.adapt(text_corpus)
    vocab = vectorizer.get_vocabulary()
    vocab_size = len(vocab)

    # Convert into stream of integers
    full_text = " ".join(text_corpus)
    full_text_ids = vectorizer([full_text])[0]

    ids_dataset = tf.data.Dataset.from_tensor_slices(full_text_ids)
    sequences = ids_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

    def split_input_target(seq):
        return seq[:-1], seq[1:]

    dataset = sequences.map(split_input_target)
    dataset = dataset.shuffle(10_000).batch(
        BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

    return dataset, vectorizer, len(vocab)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size, stateful=False):
    if stateful:
        input_layer = tf.keras.Input(batch_shape=(batch_size, None))
    else:
        input_layer = tf.keras.Input(shape=(None,))

    model = tf.keras.Sequential([
        input_layer,
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=stateful),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=stateful),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

def generate_receipe(model, vectorizer, food_name, temp=0.6):
    start_string = f"<start> title: {food_name}"
    input_ids = vectorizer([start_string])
    input_eval = input_ids

    vocab = vectorizer.get_vocabulary()
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    generated_tokens = []

    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()
    print(f"Generating recipe for {food_name}")

    for i in range(300):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # Temperature
        predictions = predictions / temp
        # Sample
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Stop token
        predicted_word = idx2word.get(predicted_id, "")
        if predicted_word == "<end>":
            break

        generated_tokens.append(predicted_word)

        # Next input
        input_eval = tf.expand_dims([predicted_id], 0)

    raw_text = " ".join(generated_tokens)
    formatted_text = raw_text.replace(" <ing> ", "\n\n[INGREDIENTS]:\n")
    formatted_text = formatted_text.replace(" <dir> ", "\n\n[DIRECTIONS]:\n")
    # Clean up spacing around punctuation
    formatted_text = formatted_text.replace(" , ", ", ").replace(" . ", ". ")

    return f"Title: {food_name}\n" + formatted_text

def main():
    # get dataset
    print(f"[INFO] - Downloading dataset from HF...")
    dataset = get_dataset()

    # format receipe
    print(f"\n[INFO] - Formatting dataset...")
    SUBSET = dataset['train'].num_rows

    print(f"\n[INFO] Processing first {SUBSET} recipes...")
    train_dataset = dataset['train'].select(range(SUBSET))
    text_corpus = [format_recipe(text) for text in train_dataset]
    print(f"\n[INFO] Text corpus created successfully.")
    print(f"Total samples: {len(text_corpus)}")

    # Show example of text_corpus
    print("\nSample Formatted Recipe")
    print(text_corpus[0])

    # Vectorization
    print(f"\n[INFO] Creating dataset...")
    dataset, vectorizer, vocab_size = create_dataset_pipeline(text_corpus)
    print("Dataset created successfully.")
    print(f"Vocab Size: {vocab_size}")

    # Build Model
    print(f"\n[INFO] - Creating training model...")
    model = build_model(
        vocab_size=vocab_size,
        embedding_dim=EMBEDDING_DIM,
        rnn_units=RNN_UNITS,
        batch_size=BATCH_SIZE
        )
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    )
    print("Model created successfully.")
    print("Check model's summary:")
    model.summary()

    # Define callbacks
    # Callbacks
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        "recipe_model.weights.h5", save_best_only=True, save_weights_only=True, monitor='loss'
    )
    early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=3, monitor='loss')

    print("[INFO] Starting Training...")
    model.fit(
        dataset, epochs=EPOCHS, callbacks=[checkpoint_cb, early_stop_cb])
    print("Training finished...")

    # Inference Model
    print(f"[INFO] - Creating inference model...")
    inference_model = build_model(
        vocab_size, EMBEDDING_DIM, RNN_UNITS, batch_size=1, stateful=True)
    inference_model.load_weights("recipe_model.weights.h5")
    inference_model.build(tf.TensorShape([1, None]))

    # Test Generation
    print(f"Generating....")
    result = generate_receipe(
        inference_model, vectorizer, "chicken pizza", temp=0.6
    )
    print("\n" + "="*40)
    print(result)
    print("="*40 + "\n")

# Execution
main()

[INFO] - Downloading dataset from HF...


Repo card metadata block was not found. Setting CardData to empty.



[INFO] - Formatting dataset...

[INFO] Processing first 6118 recipes...

[INFO] Text corpus created successfully.
Total samples: 6118

Sample Formatted Recipe
<START> title: pork chop noodle soup <ING> 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery <DIR> season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons salt and 1 teaspoon pepper . mix well and bring to a simm

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, None, 256)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, None, 512)      │     1,182,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, None, 512)      │     1,575,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 10000)    │     5,130,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,448,656 (39.86 MB)

 Trainable params: 10,448,656 (39.86 MB)

 Non-trainable params: 0 (0.00 B)

[INFO] Starting Training...
Epoch 1/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 37s 93ms/step - loss: 6.3752
Epoch 2/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 39s 104ms/step - loss: 5.9847
Epoch 3/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 43s 114ms/step - loss: 5.4265
Epoch 4/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 40s 107ms/step - loss: 4.8880
Epoch 5/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - loss: 4.4183
Epoch 6/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 110ms/step - loss: 4.0305
Epoch 7/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - loss: 3.7393
Epoch 8/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - loss: 3.5386
Epoch 9/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - loss: 3.3657
Epoch 10/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - loss: 3.2367
Epoch 11/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - loss: 3.1305
Epoch 12/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - loss: 3.0421
Epoch 13/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 41s 111ms/step - loss: 2.9683
Epoch 14/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - l

In [10]:
# قدم ۱: طراحی فرمت متن (Prompt Engineering)
# قدم ۲: تابع تبدیل داده (Data Preparation)
# قدم ۳: استانداردسازی و توکن‌سازی (Vectorization)
# قدم ۴: ساخت دیتاست (Input/Target Pipeline)
# قدم ۵: مدل و آموزش
# قدم ۶: تولید دستور پخت (Inference)